In [1]:
# Перечень команд, которые необходимо выполнить перед запуском jupyter notebook для включения решателей:
# export PATH=$PATH:/Library/Frameworks/GAMS.framework/Versions/47/Resources/
# export PATH=$PATH:/Library/Frameworks/ampl_macos64/

# Доступные решатели:
# _neos              [-] внутренняя ошибка связанная с работой со строками
# _mock_cbc          [-] попытка доступа к несуществующему полю _problem_files у ConcreteModel
# glpk               [-] не работает с нелинейными выражениями в целевой функции
# _glpk_shell        [-] не работает с нелинейными выражениями в целевой функции
# _mock_glpk         [-] попытка доступа к несуществующему полю _problem_files у ConcreteModel
# _mock_cplex        [-] не работает с нелинейными выражениями в целевой функции
# gurobi_direct      [-] не работает с нелинейными выражениями в целевой функции
# gurobi             [-] не работает с нелинейными выражениями в целевой функции
# _gurobi_shell      [-] не работает с нелинейными выражениями в целевой функции
# baron              [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# xpress             [-] не работает с нелинейными выражениями в целевой функции
# ipopt              [+] не работает с целыми значениями
# gurobi_persistent  [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gams               [+]
# _gams_shell        [+]
# xpress_direct      [-] не работает с нелинейными выражениями в целевой функции
# xpress_persistent  [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mpec_nlp           [+] не работает с целыми значениями
# mpec_minlp         [-] не работает с нелинейными выражениями в целевой функции
# appsi_gurobi       [-] не работает с нелинейными выражениями в целевой функции
# gdpopt             [-] требуется дополнительное указание алгоритма через аргумент метода solve         
# gdpopt.gloa        [-] ошибка инициализации решателя
# gdpopt.lbb         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gdpopt.loa         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gdpopt.ric         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gdpopt.enumerate   [-] не работает с нелинейными выражениями в целевой функции
# mindtpy            [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mindtpy.oa         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mindtpy.ecp        [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mindtpy.goa        [-] внутренняя ошибка работы решателя
# mindtpy.fp         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# multistart         [+] не работает с целыми значениями
# ipopt_v2           [+] не работает с целыми значениями
# gurobi_v2          [-] не работает с нелинейными выражениями в целевой функции
# gurobi_direct_v2   [-] не работает с нелинейными выражениями в целевой функции
# trustregion        [-] для инициализации требуется дополнительный аргумент degrees_of_freedom_variables
# ampl               [-] внутренняя ошибка работы решателя

In [2]:
import pyomo.environ as pyo
import pyomo.gdp as gdp
import numpy as np

In [3]:
M = 100
e = np.e

In [4]:
check_include = lambda x: (e ** x - e ** -x) / (e ** x + e ** -x)

In [5]:
check_implemented = lambda x: e ** ((x - 1) * M)

In [6]:
calculate_dependencies = lambda i, F: np.dot(F, Eff) if i == 0 else np.dot(calculate_dependencies(i - 1, F), Eff)

In [7]:
# Матрица трассируемости требований к ПО на файлы исходного кода
Erf = np.array([[  1,   0,   0,   0],  # Требование № 1 реализовано в файле № 1
                [0.5, 0.5,   0,   0],  # Требование № 2 реализовано в файлах № 1 и 2
                [  0, 0.5, 0.5,   0],  # Требования № 3 реализовано в файлах № 2 и 3
                [  0,   0, 0.5, 0.5],  # Требование № 4 реализовано в файлах № 3 и 4
                [  0,   0,   0,   1]]) # Требовнаие № 5 реализовано в файле № 4

In [8]:
# Матрица зависимостей между файлами исходного кода
Eff = np.array([[0, 1, 0, 0],  # Файл № 1 имеет зависимость на файл № 2
                [0, 0, 0, 0],  # Файл № 2 не имеет зависимостей на другие файлы
                [0, 0, 0, 0],  # Файл № 2 не имеет зависимостей на другие файлы
                [0, 0, 0, 0]]) # Файл № 2 не имеет зависимостей на другие файлы

In [9]:
# Матрица стоимостей вхождения требований в поставку
Err = np.array([
    [1, 0, 0, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 1
    [0, 1, 0, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 2
    [0, 0, 1, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 3
    [0, 0, 0, 1, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 4
    [0, 0, 0, 0, 1]  # Стоимость сопровождения функционала, если в поставку войдет требование № 5
])

In [10]:
# Число плагинов
k = 3

In [11]:
# Число требований(n) и число файлов исходного кода(m)
n, m = np.shape(Erf)

In [12]:
# Инициализация модели
model = pyo.ConcreteModel(name = 'Optimal decomposition')

In [13]:
# Матрица распределения файлов по плагинам
# Необходимо определить оптимальное распределение
# Здесь указаны ограничения на максимальное и минимальное значение
# И на то, что элементы должны быть целыми числами
set_m = pyo.Set(initialize=range(m))
set_k = pyo.Set(initialize=range(k))
model.Efp = pyo.Var(set_m, set_k, domain=pyo.Binary)

# Efp = np.array([
#     [1, 0, 0],
#     [0, 1, 0],
#     [0, 1, 0],
#     [0, 0, 1]
# ])


In [14]:
# Ограничения на сумму элементов в строках
model.constraints = pyo.ConstraintList()
equations = []
for row in np.array(model.Efp):
    model.constraints.add((sum(row) == 1))

In [15]:
R = np.array([1, 0, 0, 0, 0])

In [16]:
F = np.dot(R, Erf)

In [17]:
files_count = len(F)

In [18]:
for i in range(files_count):
    dependencies = calculate_dependencies(i, F)
    F += dependencies

In [19]:
F

array([1., 1., 0., 0.])

In [20]:
P = np.dot(F, model.Efp)

In [21]:
F_d = np.dot(model.Efp, P)

In [22]:
F_d = check_include(M * F_d)

In [23]:
R_d = np.dot(Erf, F_d)

In [24]:
R_d = check_implemented(R_d)

In [25]:
costs = np.dot(R_d, Err)

In [26]:
model.OBJ = pyo.Objective(expr = sum(costs), sense=pyo.minimize)

In [27]:
model.pprint()

1 Var Declarations
    Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :  None :     1 : False :  True : Binary
        (0, 1) :     0 :  None :     1 : False :  True : Binary
        (0, 2) :     0 :  None :     1 : False :  True : Binary
        (1, 0) :     0 :  None :     1 : False :  True : Binary
        (1, 1) :     0 :  None :     1 : False :  True : Binary
        (1, 2) :     0 :  None :     1 : False :  True : Binary
        (2, 0) :     0 :  None :     1 : False :  True : Binary
        (2, 1) :     0 :  None :     1 : False :  True : Binary
        (2, 2) :     0 :  None :     1 : False :  True : Binary
        (3, 0) :     0 :  None :     1 : False :  True : Binary
        (3, 1) :     0 :  None :     1 : False :  True : Binary
        (3, 2) :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Sense    

In [28]:
gams = pyo.SolverFactory('gams')

In [29]:
gams_instance = model.create_instance()

In [30]:
gams_result = gams.solve(gams_instance)

In [31]:
gams_instance.Efp.display()

Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :   0.0 :     1 : False : False : Binary
    (0, 1) :     0 :   1.0 :     1 : False : False : Binary
    (0, 2) :     0 :   0.0 :     1 : False : False : Binary
    (1, 0) :     0 :   1.0 :     1 : False : False : Binary
    (1, 1) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2) :     0 :   0.0 :     1 : False : False : Binary
    (2, 0) :     0 :   0.0 :     1 : False : False : Binary
    (2, 1) :     0 :   1.0 :     1 : False : False : Binary
    (2, 2) :     0 :   0.0 :     1 : False : False : Binary
    (3, 0) :     0 :   1.0 :     1 : False : False : Binary
    (3, 1) :     0 :   0.0 :     1 : False : False : Binary
    (3, 2) :     0 :   0.0 :     1 : False : False : Binary


In [32]:
_gams_shell = pyo.SolverFactory('_gams_shell')

In [33]:
_gams_shell_instance = model.create_instance()

In [34]:
_gams_shell_result = _gams_shell.solve(_gams_shell_instance)

In [35]:
_gams_shell_instance.Efp.display()

Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :   0.0 :     1 : False : False : Binary
    (0, 1) :     0 :   1.0 :     1 : False : False : Binary
    (0, 2) :     0 :   0.0 :     1 : False : False : Binary
    (1, 0) :     0 :   1.0 :     1 : False : False : Binary
    (1, 1) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2) :     0 :   0.0 :     1 : False : False : Binary
    (2, 0) :     0 :   0.0 :     1 : False : False : Binary
    (2, 1) :     0 :   1.0 :     1 : False : False : Binary
    (2, 2) :     0 :   0.0 :     1 : False : False : Binary
    (3, 0) :     0 :   1.0 :     1 : False : False : Binary
    (3, 1) :     0 :   0.0 :     1 : False : False : Binary
    (3, 2) :     0 :   0.0 :     1 : False : False : Binary


In [36]:
ipopt = pyo.SolverFactory('ipopt')

In [37]:
ipopt_instance = model.create_instance()

In [38]:
ipopt_result = ipopt.solve(ipopt_instance)

In [39]:
ipopt_instance.Efp.display()

Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value               : Upper : Fixed : Stale : Domain
    (0, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (0, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (0, 2) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (1, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (1, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (1, 2) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (2, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (2, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (2, 2) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (3, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (3, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (3, 2) :     0 : 0.33333333333333337 :     1 : False : False : B

In [40]:
mpec_nlp = pyo.SolverFactory('mpec_nlp')

In [41]:
mpec_nlp_instance = model.create_instance()

In [42]:
mpec_nlp_result = mpec_nlp.solve(mpec_nlp_instance)

In [43]:
mpec_nlp_instance.Efp.display()

Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value               : Upper : Fixed : Stale : Domain
    (0, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (0, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (0, 2) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (1, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (1, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (1, 2) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (2, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (2, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (2, 2) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (3, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (3, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (3, 2) :     0 : 0.33333333333333337 :     1 : False : False : B

In [44]:
multistart = pyo.SolverFactory('multistart')

In [45]:
multistart_instance = model.create_instance()

In [46]:
multistart_result = multistart.solve(multistart_instance)

In [47]:
multistart_instance.Efp.display()

Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value               : Upper : Fixed : Stale : Domain
    (0, 0) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (0, 1) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (0, 2) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (1, 0) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (1, 1) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (1, 2) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (2, 0) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (2, 1) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (2, 2) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (3, 0) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (3, 1) :     0 : 0.33333333333333337 :     1 : False :  True : Binary
    (3, 2) :     0 : 0.33333333333333337 :     1 : False :  True : B

In [48]:
ipopt_v2 = pyo.SolverFactory('ipopt_v2')

In [49]:
ipopt_v2_instance = model.create_instance()

In [50]:
ipopt_v2_result = ipopt_v2.solve(ipopt_v2_instance)

In [51]:
ipopt_v2_instance.Efp.display()

Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value               : Upper : Fixed : Stale : Domain
    (0, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (0, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (0, 2) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (1, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (1, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (1, 2) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (2, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (2, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (2, 2) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (3, 0) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (3, 1) :     0 : 0.33333333333333337 :     1 : False : False : Binary
    (3, 2) :     0 : 0.33333333333333337 :     1 : False : False : B

In [52]:
from pyomo.contrib.latex_printer import latex_printer
pstr = latex_printer(model.OBJ)